In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

In [2]:
r = requests.get('https://www.forbes.com/lists/global2000/?sh=3a43fc785ac0')
html = BS(r.content, 'html.parser')

In [3]:
c1 = html.find_all(class_="rank first table-cell rank")
c2 = html.find_all(class_="organizationName second table-cell name")
c3 =html.find_all(class_="country table-cell country")
c4 =html.find_all(class_="revenue table-cell sales")
c5 =html.find_all(class_="profits table-cell profit")
c6 =html.find_all(class_="assets table-cell assets")
c7 =html.find_all(class_="marketValue table-cell market value")

In [4]:
c1text = [i.text for i in c1]
c2text = [i.text for i in c2]
c3text = [i.text for i in c3]
c4text = [i.text for i in c4]
c5text = [i.text for i in c5]
c6text = [i.text for i in c6]
c7text = [i.text for i in c7]

In [7]:
df = pd.DataFrame({'rank': c1text, 'name': c2text, 'country': c3text, 'sales': c4text, 'profit': c5text, 'assets': c6text})

In [8]:
df = df.drop(df[df.iloc[:, 1] == 'Pfizer'].index)
df = df.drop(df[df.iloc[:, 1] == 'The Home Depot'].index)
df = df.drop(df[df.iloc[:, 1] == 'Siemens'].index)
df = df.drop(df[df.iloc[:, 1] == 'Waste Management'].index)
df = df.drop(df[df.iloc[:, 1] == 'Fifth Third Bank'].index)
df = df.drop(df[df.iloc[:, 1] == 'Universal Music Group'].index)
df = df.drop(df[df.iloc[:, 1] == 'Eurofins Scientific Societe Europeenne'].index)
df = df.drop(df[df.iloc[:, 1] == 'Zhongliang Holdings'].index)
df = df.drop(df[df.iloc[:, 1] == 'Epiroc'].index)
df = df.drop(df[df.iloc[:, 1] == 'CoStar Group'].index)
df = df.drop(df[df.iloc[:, 1] == 'XPO'].index) #это я удалила, потому что у них может измениться маркет вэлью (на сайте так сказано)

In [9]:
df.insert(6,'MV', c7text)
df.head()

,rank,name,country,sales,profit,assets,MV
0,1.,Berkshire Hathaway,United States,$276.09 B,$89.8 B,$958.78 B,$741.48 B
1,2.,ICBC,China,$208.13 B,$54.03 B,"$5,518.51 B",$214.43 B
2,3.,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia,$400.38 B,$105.36 B,$576.04 B,"$2,292.08 B"
3,4.,JPMorgan Chase,United States,$124.54 B,$42.12 B,"$3,954.69 B",$374.45 B
4,5.,China Construction Bank,China,$202.07 B,$46.89 B,"$4,746.95 B",$181.32 B


In [10]:
df.index = [int(i) for i in range(1990)]

In [11]:
d = html.find_all('a')
hrefs = []
for i in d:
        h = i.get('href')
        if h!=None:
            hrefs.append(h)
hrefs = hrefs[9:1999]

In [ ]:
BigList = [] #для каждой компании парсим лист ["Отрасль", " Год основания", "Город", "Страна", "CEO", "Количество работников"]
for el in hrefs:
    r = requests.get(f'{el}')
    html = BS(r.content, 'html.parser')
    f = html.find_all('span',class_='profile-stats__text')
    l=[]
    for i in f:
        if i.text not in l:
            l.append(i.text)
    if len(l)==5:
        l = [None]+l
    BigList.append(l)

In [ ]:
share_prices = []#парсим цены акций, если акции не торгуются, то пишем None
for i in hrefs:
    r = requests.get(f'{i}')
    html = BS(r.content, 'html.parser')
    price = html.find(class_='profile-info__item-value')
    if price != None:
        share_prices.append(price.text)
    else: 
        share_prices.append('None')

In [ ]:
df2 = pd.DataFrame(BigList, 
columns=['Industry','Founded', 'Headquaters', 'Country2', 'CEO', 'Employees'], index = [int(i) for i in range(1990)])
df2.insert(6, 'Share price', share_prices)
result = pd.concat([df,df2], axis=1)
result

,rank,name,country,sales,profit,assets,MV,Industry,Founded,Headquaters,Country2,CEO,Employees,Share price
0,1.,Berkshire Hathaway,United States,$276.09 B,$89.8 B,$958.78 B,$741.48 B,Diversified Financials,1939,"Omaha, Nebraska",United States,Warren Edward Buffett,"372,000",$318.05
1,2.,ICBC,China,$208.13 B,$54.03 B,"$5,518.51 B",$214.43 B,None,1984,Beijing,China,Liao Lin,"434,089",None
2,3.,Saudi Arabian Oil Company (Saudi Aramco),Saudi Arabia,$400.38 B,$105.36 B,$576.04 B,"$2,292.08 B",Oil & Gas Operations,1933,Dhahran,Saudi Arabia,Amin bin Hasan Al-Nasser,"68,493",None
3,4.,JPMorgan Chase,United States,$124.54 B,$42.12 B,"$3,954.69 B",$374.45 B,Banking,2000,"New York, New York",United States,Jamie Dimon,"270,000",$128.99
4,5.,China Construction Bank,China,$202.07 B,$46.89 B,"$4,746.95 B",$181.32 B,None,1954,Beijing,China,Jinliang Zhang,"347,319",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1985,1995.,Shenzhen Feima International Supply Chain,China,$37 M,$1.41 B,$166 M,$1.14 B,Business Services & Supplies,1998,Shenzhen,China,Shan Min Huang,306,None
1986,1997.,NMDC,India,$3.52 B,$1.41 B,$5.71 B,$6.4 B,Materials,1958,Hyderabad,India,Sumit Deb,"16,944",None
1987,1997.,Sichuan Changhong Electric,China,$15.72 B,$53.1 M,$12.11 B,$1.96 B,Consumer Durables,1988,Mianyang,China,Wei Li,"51,841",None
1988,1999.,Satellite Chemical,China,$4.41 B,$931.3 M,$7.64 B,$9.52 B,Chemicals,China,"3,409",None,None,None,None


In [ ]:
#убираем строки, в которых мало инфы
ind =[]
for i in BigList:
    if len(i)<6:
        ind.append(BigList.index(i))
print(set(ind))

{519, 1544, 1933, 1935, 1041, 1809, 1688, 1690, 1307, 1958, 1836, 813, 1840, 946, 1465, 1339, 576, 321, 1729, 1984, 1988, 209, 1364, 1509, 489, 363, 1899, 240, 1776, 1396, 248, 1658}


In [ ]:
result = result.drop(index = set(ind))

In [ ]:
f = result['Founded'].tolist()
ind = []
for i in f:
    if i.isdigit()!=True:
        ind.append(i)
for i in ind:
    result = result.drop(result[result.iloc[:, 8] == f'{i}'].index)

In [ ]:
result = result.drop(result[result.iloc[:, 8] == 'Engineering, Manufacturing'].index)

In [ ]:
result = result.drop(columns = ['Country2'])

In [ ]:
result.to_csv('forbes2000 (1).csv', sep=',', index=False, lineterminator='')